# VWAP with Tick Data v1

## Parameters
Parameters: &lt;side&gt; &lt;quantity&gt; [limit_px]  
side = "b" (buy) or "s" (sell)  
quantity = # of shares  
limit_px = limit price (optional)  

In [ ]:
%matplotlib inline

import time
import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

matplotlib.rcParams[ 'figure.figsize' ] = ( 14, 6 )

def show_time( label_string ):
    ts = time.time()
    st = datetime.datetime.fromtimestamp( ts ).strftime( '%Y-%m-%d %H:%M:%S:%f' )
    print( label_string + ' : ' + st )

In [ ]:
vwap_coefs = [7.81292630e-03, -6.23749840e-05, 3.13654288e-07, -7.63471095e-10, 7.19639892e-13]

# use coefs to make our target series
def vwap_target( bar_num, coefs ):
    return ( coefs[ 0 ] * bar_num + 
             coefs[ 1 ] * bar_num**2 + 
             coefs[ 2 ] * bar_num**3 +
             coefs[ 3 ] * bar_num**4 +
             coefs[ 4 ] * bar_num**5 )

### Initialization

In [ ]:
# get our market data
# parsing of our pre-saved data is much simpler. We could also include 
taq = pd.read_csv( '../data/AAPL_20180117.gz', index_col = 0, parse_dates = True )

# set order parameters
order_side = "s"
order_quantity = 250000
# EFTR
# order_limit_price = 10.00

In [ ]:
# generate target schedule - use bins 1 - 390 giving an automatic 1 minute "look ahead"
# note that targets have been converted to shares from percent
order_targets = vwap_target( np.arange( 0, 391, dtype='int64' ), np.array(vwap_coefs) ) * order_quantity

In [ ]:
type(np.array(vwap_coefs))

In [ ]:
type(np.arange( 0, 391 ))

In [ ]:
_ = plt.plot(order_targets)

In [ ]:
# check our data
taq.head()

In [ ]:
# cut our data to market hours
trading_day = taq['2018-01-17 09:30:00':'2018-01-17 16:00:00']

In [ ]:
trading_day.head()

In [ ]:
len(trading_day)

In [ ]:
# let's set up a container to hold trades. preinitialize with the index
trades = pd.DataFrame( columns = [ 'price' , 'shares', 'bar', 'type' ], index=trading_day.index )

### Event Loop

In [ ]:
# main event loop
current_bar = 1
current_target = 0
quantity_filled = 0
quantity_remaining = order_quantity - quantity_filled
threshold_value = 5000
bid_price = 0.0
bid_quantity = 0.0
offer_price = 0.0
offer_quantity = 0.0
last_price = 0.0
last_quantity = 0.0
total_volume = 0.0
vwap_numerator = 0.0
new_trade_price = 0.0
new_trade_quantity = 0

show_time( 'start loop' )
for index, row in trading_day.iterrows():
    # figure out which minute we are in after the open
    time_from_open = ( index - pd.Timedelta( hours = 9, minutes = 30 ) )
    minutes_from_open = ( time_from_open.hour * 60 ) + time_from_open.minute
        
    # determine message type
    if ( pd.isnull( row[ 'Trade Price' ] ) ): # it's a quote
        # collect BBO
        bid_price = row[ 'Bid_Price' ]
        offer_price = row[ 'Offer_Price' ]
        
        # get price and adjust for lot size
        bid_quantity = row[ 'Bid_Size' ] * 100 
        offer_quantity = row[ 'Offer_Size' ] * 100
    
    else: # it's a trade
        last_price = row[ 'Trade Price' ]
        last_quantity = row[ 'Trade Volume' ]
        
        # collect volume
        total_volume += last_quantity
        
        # collect rolling VWAP
        vwap_numerator += last_quantity * last_price

    # are we in a new minute?
    if ( minutes_from_open > current_bar ):
        # we're in a new bar do new bar things here
        current_bar = minutes_from_open
        current_target = min( np.ceil( order_targets[ current_bar ] ), order_quantity )
        #print ( str(current_bar) + ': ' + str( current_target ) )
        
    # trade logic
    # v1: are we behind the target at all? If so, do a trade, just pay the spread
    quantity_behind = current_target - quantity_filled
    if ( quantity_behind > threshold_value ): # we need to trade 
        # assume you can execute the full size across spread
        new_trade_quantity = quantity_behind
        if ( order_side == 'b' ):
            new_trade_price = offer_price
        else: 
            new_trade_price = bid_price     
        trades.loc[ index ] = [ new_trade_price, new_trade_quantity, current_bar, "aggressive" ]
        #print("cross spread: {} {} shares @ {} in bar {}".format(order_side, new_trade_quantity, new_trade_price, current_bar))
    else: #we're not very behind, assume we can trade at the bid / offer
        new_trade_quantity = quantity_behind
        if ( order_side == 'b' ):
            new_trade_price = bid_price
        else: 
            new_trade_price = offer_price
        trades.loc[ index ] = [ new_trade_price, new_trade_quantity, current_bar, "passive" ]
        #print("passive trade: {} {} shares @ {} in bar {}".format(order_side, new_trade_quantity, new_trade_price, current_bar))
        # update quantity remaining
        quantity_remaining = min( 0, quantity_remaining - new_trade_quantity )
        quantity_filled += new_trade_quantity    
        
# looping done
show_time( 'end loop' )

In [ ]:
# incidentally... trying to record into dataframe using this code... gave up after 40 minutes...
# new_row = pd.DataFrame( [[ new_trade_price, new_trade_quantity, order_side ]], 
#                              columns = [ 'fill_price' , 'fill_quantity', 'fill_side' ],
#                              index = [ index ] )
# trades = pd.concat( [ trades, pd.DataFrame( new_row ) ], ignore_index = False )

In [ ]:
# Now, let's look at some stats
trades = trades.dropna()
day_vwap = vwap_numerator / total_volume
print( 'stock volume: ' + str( total_volume ) )
print( 'order shares traded: ' + str( quantity_filled ) )
avg_price = (trades[ 'price' ] * trades[ 'shares' ]).sum() / trades[ 'shares' ].sum()
print( 'average price: ' + str( avg_price ) )
print( 'day vwap: ' + str( day_vwap ) )
print( 'average trade size: ' + str( trades[ 'shares' ].mean() ) )

In [ ]:
trades[ 'accum_shares' ] = trades[ 'shares' ].cumsum()

In [ ]:
plt.plot(trades['accum_shares'])

In [ ]:
plt.plot(trades['price'])
plt.title('realized trade prices')

In [ ]:
# zero price trades?
trades[trades['price'] == 0]

In [ ]:
trading_day['2018-01-17 09:30:00']

In [ ]:
all_trades = trading_day[[ 'Trade Volume', 'Trade Price' ]].copy()
all_trades = all_trades.dropna()

In [ ]:
plt.plot( all_trades['Trade Price'] )

In [ ]:
# for now, let's ignore the extreme values and plot these together

day_plot = plt.plot( all_trades[ 'Trade Price' ], label = 'stock chart' )
trade_plot = plt.plot( trades[ 'price' ], label = 'Our Trade' )
plt.title( 'Our trades vs. the day' )
plt.ylim( 174, 180 )
plt.legend()